# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('df_unique', engine)
X = df['message']
y = df.drop(['message','id','genre','original'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=y_test.columns)
for column in y_test.columns:
    print(f"Classification report for {column}:")
    print(classification_report(y_test[column], y_pred[column]))
    print("\n")

Classification report for related:
             precision    recall  f1-score   support

          0       0.65      0.47      0.54      1860
          1       0.85      0.92      0.88      5961
          2       0.36      0.35      0.36        54

avg / total       0.80      0.81      0.80      7875



Classification report for request:
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      6538
          1       0.80      0.43      0.56      1337

avg / total       0.88      0.88      0.87      7875



Classification report for offer:
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      7834
          1       0.50      0.02      0.05        41

avg / total       0.99      0.99      0.99      7875



Classification report for aid_related:
             precision    recall  f1-score   support

          0       0.76      0.86      0.80      4605
          1       0.75      0.61      0.67     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
# Define parameters for grid search
parameters = {
    'clf__estimator__n_estimators': [50, 100]
    #'clf__estimator__min_samples_split': [2, 4]
}

# Create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters,verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Fit grid search
cv.fit(X_train, y_train)

# Print best parameters
print("Best parameters:", cv.best_params_)

# Use best estimator for predictions
y_pred = cv.best_estimator_.predict(X_test)

# Convert y_pred to DataFrame
y_pred = pd.DataFrame(y_pred, columns=y_test.columns)

# Print classification report for each column
for column in y_test.columns:
    print(f"Classification report for {column}:")
    print(classification_report(y_test[column], y_pred[column]))
    print("\n")

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__n_estimators=50 .................................
[CV] .................. clf__estimator__n_estimators=50, total= 1.8min
[CV] clf__estimator__n_estimators=50 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.0min remaining:    0.0s


[CV] .................. clf__estimator__n_estimators=50, total= 1.8min
[CV] clf__estimator__n_estimators=50 .................................
[CV] .................. clf__estimator__n_estimators=50, total= 1.8min
[CV] clf__estimator__n_estimators=100 ................................
[CV] ................. clf__estimator__n_estimators=100, total= 3.5min
[CV] clf__estimator__n_estimators=100 ................................
[CV] ................. clf__estimator__n_estimators=100, total= 3.5min
[CV] clf__estimator__n_estimators=100 ................................
[CV] ................. clf__estimator__n_estimators=100, total= 3.6min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 17.9min finished


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Custom transformer to extract additional features
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return pd.DataFrame({
            'text_length': X.apply(len),
            'word_count': X.apply(lambda x: len(x.split())),
            'unique_word_count': X.apply(lambda x: len(set(x.split())))
        })

# Define the pipeline
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('text_length', TextLengthExtractor())
    ])),
    ('clf', MultiOutputClassifier(
        RandomForestClassifier(random_state=42)
    ))
])

# Define parameters for grid search
parameters = {
    'features__text_pipeline__vect__max_df': (0.5, 0.75),
    'features__text_pipeline__vect__max_features': (5000, 10000),
    'clf__estimator__n_estimators': [100, 200],
    'clf__estimator__min_samples_split': [2, 4]
}

# Create and run grid search
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, n_jobs=-1, verbose=2)
cv.fit(X_train, y_train)

# Print best parameters
print("Best parameters:", cv.best_params_)

# Use best estimator for predictions
y_pred = cv.best_estimator_.predict(X_test)

# Convert y_pred to DataFrame
y_pred = pd.DataFrame(y_pred, columns=y_test.columns)

# Print classification report for each column
for column in y_test.columns:
    print(f"Classification report for {column}:")
    print(classification_report(y_test[column], y_pred[column]))
    print("\n")

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=5000 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=5000 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=5000 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=10000 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=10000 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, features__text_p

### 9. Export your model as a pickle file

In [ ]:
import pickle

# Assuming 'cv.best_estimator_' is yourtt best model after grid search
best_model = cv.best_estimator_

# Save the model to a pickle file
with open('disaster_response_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

print("Model saved as 'disaster_response_model.pkl'")

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.